# APLICACIONES EN CIENCIAS DE COMPUTACION

## Laboratorio 3: Operadores de Algoritmos Geneticos para resolver el problema de las n-reinas

Abajo puede encontrar la implementacion de un algoritmo genético para resolver el problema de las n-reinas. El algoritmo ya esta implementado y funcionando con algunos operadores basicos. La clase Individual_nqueens implementa el individuo para este problema. Tiene 2 atributos: chromosome y fitness. Cromosome es una lista, donde el elemento i indica la posicion de la reina en la columna i. El fitness es el nro de pares de reinas que no se atacan y se puede obtener llamando a la funcion fitness_nqueens(). La clase individuo también implementa el operador de cruzamiento de un punto (crossover_onepoint) y el operador de mutacion swap (mutate_swap). 

La tarea de este laboratorio consiste en implementar en la clase Individual_nqueensen dos nuevos operadores (marcados con #TODO): el operador de cruzamiento uniforme (crossover_uniform) y el operador de mutación que cambia aleatoriamente la posicion de una reina (mutate_position).

Al final de este notebook puede encontrar las tareas y cuestiones que seran evaluadas en este laboratorio.

<b>Clase abstracta de un individuo de algoritmo genético (no modificar)</b>

In [2]:
class Individual:
    "Clase abstracta para individuos de un algoritmo evolutivo."

    def __init__(self, chromosome):
        self.chromosome = chromosome

    def crossover(self, other):
        "Retorna un nuevo individuo cruzando self y other."
        raise NotImplementedError
        
    def mutate(self):
        "Cambia los valores de algunos genes."
        raise NotImplementedError        

<b>Clase concreta de un individuo del problema de las n-reinas</b> (Falta implementar crossover_uniform y mutate_position)

In [36]:
import random

class Individual_nqueens(Individual):
    "Clase que implementa el individuo en el problema de las n-reinas."

    def __init__(self, chromosome):
        self.chromosome = chromosome[:]
        self.fitness = -1

    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre self y other "
        c = random.randrange(len(self.chromosome))
        ind1 = Individual_nqueens(self.chromosome[:c] + other.chromosome[c:])
        ind2 = Individual_nqueens(other.chromosome[:c] + self.chromosome[c:])
        return [ind1, ind2]   
    
    def crossover_uniform(self, other):
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        #TODO
      
        ind1=[]
        ind2=[]
        
        for i in range(0,len(self.chromosome)):            
            if (round(random.random())==0):
                ind1.append(self.chromosome[i])
                ind2.append(other.chromosome[i])
            else:
                ind1.append(other.chromosome[i])
                ind2.append(self.chromosome[i])
                
        ind1=Individual_nqueens(ind1)
        ind2=Individual_nqueens(ind2)
        
        return [ind1, ind2]
          

    def mutate_position(self):
        "Cambia aleatoriamente la posicion de una reina."
        #TODO
        #Debemos de conseguir una posición aleatoria
        i= random.randrange(len(self.chromosome))
        
        mutated_ind = Individual_nqueens(self.chromosome[:])
        
        mutated_ind.chromosome[i]= random.randrange(len(self.chromosome))+1
        
        return mutated_ind
            
        
    def mutate_swap(self):
        "Intercambia la posicion de dos genes."
        mutated_ind = Individual_nqueens(self.chromosome[:])
        indexOne = random.randint(0,len(mutated_ind.chromosome)-1)
        indexTwo = random.randint(0,len(mutated_ind.chromosome)-1)
        temp = mutated_ind.chromosome[indexOne]
        mutated_ind.chromosome[indexOne] = mutated_ind.chromosome[indexTwo]
        mutated_ind.chromosome[indexTwo] = temp
        return mutated_ind

<b>Funcion de fitness para evaluar un individuo del problema de las n-reinas</b>

In [37]:
def fitness_nqueens(chromosome):
    """Retorna el fitness de un cromosoma en el problema de las n-reinas (nro de pares de reinas no atacadas) """
    n = len(chromosome)  # nro de reinas
    fitness = 0
    for i in range(n-1):
        for j in range(i+1, n):
            # si el par de reinas i, j  no etan en la misma fila o diagonales => par no atacado
            if chromosome[j] not in [chromosome[i], chromosome[i] - (j-i),  chromosome[i] + (j-i)]:
                fitness = fitness + 1
    return fitness

<b>Funcion para evaluar toda una población de individuos con la funcion de fitnes especificada</b>

In [38]:
def evaluate_population(population, fitness_fn):
    """ Evalua una poblacion de individuos con la funcion de fitness pasada """
    popsize = len(population)
    for i in range(popsize):
        if population[i].fitness == -1:    # si el individuo no esta evaluado
            population[i].fitness = fitness_fn(population[i].chromosome)

<b>Funcion que selecciona con el metodo de la ruleta un par de individuos de population para cruzamiento </b>

In [39]:
def select_parents_roulette(population):
    popsize = len(population)
    
    # Escoje el primer padre
    sumfitness = sum([indiv.fitness for indiv in population])  # suma total del fitness de la poblacion
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent1 = i
            break
     
    # Escoje el segundo padre, desconsiderando el primer padre
    sumfitness = sumfitness - population[iParent1].fitness # retira el fitness del padre ya escogido
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        if i == iParent1: continue   # si es el primer padre 
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent2 = i
            break        
    return (population[iParent1], population[iParent2])

<b>Funcion que selecciona sobrevivientes para la sgte generacion, dada la poblacion actual y poblacion de hijos </b>

In [40]:
def select_survivors(population, offspring_population, numsurvivors):
    next_population = []
    population.extend(offspring_population) # une las dos poblaciones
    isurvivors = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:numsurvivors]
    for i in range(numsurvivors): next_population.append(population[isurvivors[i]])
    return next_population

<b>Algoritmo Genetico</b>   
Recibe una poblacion inicial, funcion de fitness, numero de generaciones (ngen) y taza de mutación (pmut)

In [67]:
def genetic_algorithm(population, fitness_fn, ngen=100, pmut=0.1):
    "Algoritmo Genetico "
    
    popsize = len(population)
    evaluate_population(population, fitness_fn)  # evalua la poblacion inicial
    ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
    bestfitness = [population[ibest[0]].fitness]
    print("Poblacion inicial, best_fitness = {}".format(population[ibest[0]].fitness))
    
    for g in range(ngen):   # Por cada generacion
        
        ## Selecciona las parejas de padres para cruzamiento 
        mating_pool = []
        for i in range(int(popsize/2)): mating_pool.append(select_parents_roulette(population)) 
        
        ## Crea la poblacion descendencia cruzando las parejas del mating pool 
        offspring_population = []
        for i in range(len(mating_pool)): 
            #offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]) ) # cruzamiento 1 punto
            offspring_population.extend( mating_pool[i][0].crossover_uniform(mating_pool[i][1]) ) # cruzamiento uniforme

        ## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
        for i in range(len(offspring_population)):
            if random.uniform(0, 1) < pmut: 
                #offspring_population[i] = offspring_population[i].mutate_swap()        # mutacion swap
                offspring_population[i] = offspring_population[i].mutate_position()   # mutacion de una posicion
        
        ## Evalua la poblacion descendencia creada
        evaluate_population(offspring_population, fitness_fn)  # evalua la poblacion inicial
        
        ## Selecciona popsize individuos para la sgte. generación de la union de la pob. actual y  pob. descendencia
        population = select_survivors(population, offspring_population, popsize)

        ## Almacena la historia del fitness del mejor individuo
        ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
        bestfitness.append(population[ibest[0]].fitness)
        print("generacion {}, best_fitness = {}".format(g, population[ibest[0]].fitness))
    
    return population[ibest[0]], bestfitness  

 <b>Algoritmo de Busqueda Genetica para el problema de las n-reinas</b>   

In [68]:
def genetic_search_nqueens(fitness_fn, num_queens=10, popsize=10, ngen=100, pmut=0.1):
    import random
    population = []

    ## Crea la poblacion inicial con cromosomas aleatorios
    #for i in range(popsize):
    #    chromosome = [j for j in range(1,num_queens+1)]
    #    random.shuffle(chromosome)
    #    population.append( Individual_nqueens(chromosome) )
        
    ## Crea la poblacion inicial con los siguientes cromosomas    
    chromosomes = [[1,3,1,3,1,3,1,3,1,3],
                   [2,4,2,4,2,4,2,4,2,4],
                   [3,5,3,5,3,5,3,5,3,5],
                   [4,6,4,6,4,6,4,6,4,6],
                   [5,7,5,7,5,7,5,7,5,7],
                   [6,8,6,8,6,8,6,8,6,8],
                   [7,9,7,9,7,9,7,9,7,9],
                   [8,10,8,10,8,10,8,10,8,10],
                   [9,1,9,1,9,1,9,1,9,1],
                   [10,2,10,2,10,2,10,2,10,2] ]   
    for i in range(popsize):
        population.append( Individual_nqueens(chromosomes[i]) )   
        
    ## llama al algoritmo genetico para encontrar una solucion al problema de las n reinas
    return genetic_algorithm(population, fitness_fn, ngen, pmut)        

# Tareas y cuestiones:

<b>1) Comparar el desempeño de los operadores de cruzamiento (8 puntos)</b> 

Aqui vamos a comparar el desempeño de los 2 operadores de cruzamiento implementados. Para ello probaremos en el problema de 10 reinas, 10 individuos y taza de mutación = 0 (a fin de centrarnos solo en el efecto de cruzamiento). Ejecute 10 veces el codigo abajo con crossover_onepoint (registrando el best_fitness de la generacion 99 en cada ejecución) y luego calcule la media de los valores registrados. Repita el experimento con crossover_uniform (comente la linea de crossover_onepoint y descomente la linea de crossover_uniform  en genetic_algorithm() ). Note que la funcion genetic_search_nqueens tiene una población inicial predefinida. Esto se ha hecho para descontar también la variabilidad de la poblacion inicial, asi solo la calidad de los operadores estaria influyendo en los resultados. 

In [56]:
import matplotlib.pyplot as plt
best_ind, bestfitness = genetic_search_nqueens(fitness_nqueens, 10, 10, 100, 0.0)
plt.plot(bestfitness)
plt.show()

<b> - ¿Cual de los dos operadores de cruzamiento consigue en media mejores resultados? </b> Intente dar una justificativa al respecto.       

Crossover one point:
Lista con los 10 mejores fitness de muestra (Generación 99) = [37,42,39,39,42,40,39,41,40,40].
Media= 39.9

Crossover uniform:
Lista con los 10 mejores fitness de muestra (Generación 99) = [41,42,40,42,42,43,40,41,42,39].
Media= 41.2

<b> El operador de cruzamiento uniforme da una media mayor, por lo que consigue un mejor fitness en la muestra. </b>

 La justificativa que se puede ver, es que mientras que el cruzamiento en un punto genera hijos parcialmente diferentes (un intercambio de cromosomas en bloque) y genera hijos con menos diversidad; el cruzamiento uniforme es aleatorio y genera una diferente estructura y más diversa en los individuos.

<b>2) Buscar la mejor combinación de operadores (12 puntos)</b> 

Ahora deberá identificar el mejor operador de mutación para trabajar junto con el operador de cruzamiento indentificado en la tarea anterior. Para ello deberá realizar 2 experimentos similares a la tarea 1: deje fijo el operador de cruzamiento identificado en la tarea 1 y experimente cada operador de mutacion por separado (ejecutando 10 veces el codigo abajo). La taza de mutación a probar es pmut=0.25 (mutacion sucede en aprox. un 25 % de los hijos). 

In [79]:
best_ind, bestfitness = genetic_search_nqueens(fitness_nqueens, 10, 10, 100, 0.25)
plt.plot(bestfitness)
plt.show()

Crossover uniform + mutación swap:
Lista con los 10 mejores fitness de muestra (Generación 99) = [43,45,41,43,41,40,42,43,43,42].
Media= 42.3

Crossover uniform + mutación position:
Lista con los 10 mejores fitness de muestra (Generación 99) = [44,43,44,44,43,44,42,43,43,44].
Media= 43.4

<b> - ¿Los resultados con los operadores de mutación son mejores que con solo cruzamiento? porque? </b>

Sí son mejores, en ambos casos. Esto se debe a que tras el cruce, los hijos salen diversos al estado inicial y esto es maximizado al mutar tras su concepción.

<b> - ¿Con cuál de los dos operadores de mutación se consigue en media mejores resultados? </b> Intente dar una justificativa al respecto.      

Se consigue los mejores resultados al combinar el cruzamiento uniforme con la mutación por posición. Esto se debe a que mientras que un swap simplemente hace un cambio entre dos valores ya existentes; la mutación por posición genera un cambio mucho más diverso en cuanto a posiciones no existentes en los cromosomas iniciales.